In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import re
from tqdm import tqdm

### TODO: Do this in a jupyter file instead

# Merge the simple CSVs based on app_id
gamesdf = pd.read_csv("./games.csv", quotechar='"', escapechar='\\', dtype="str")
genresdf = pd.read_csv("./genres.csv", quotechar='"', escapechar='\\', dtype="str")
gamesdf = pd.merge(gamesdf, genresdf, on="app_id")
reviewsdf = pd.read_csv("./reviews.csv", quotechar='"', escapechar='\\', dtype="str")
gamesdf = pd.merge(gamesdf, reviewsdf, on="app_id")

In [83]:
### NA values are written as 'N'. Let us fix this below

gamesdf = gamesdf.map(lambda x: np.nan if x == "N" else x)


### The languages column has <strong> tags around * symbols in it sometimes. Let's get rid of these.

gamesdf["languages"] = gamesdf["languages"].str.replace("<strong>*</strong>", "")


### Those asterixes correspond to having full audio support for those languages.
# We don't really care about this so let's get rid of that too
gamesdf["languages"] = gamesdf["languages"].str.replace("<br>languages with full audio support", "")


### Used to check if values are converted correctly. No rows should display if all values with 'N' are properly converted to NaN
# matches = gamesdf == 'N'
# display(gamesdf[matches.any(axis=1)])





In [84]:
### Some columns aren't typed properly. For example is_free is typed as an object instead of a boolean.

# Let's convert the release date to datetimes
gamesdf["release_date"] = pd.to_datetime(gamesdf["release_date"])

# Now let's convert the is_free column. It uses 0's and 1's to represent false or true respectively
# Let's map that to a boolean
gamesdf["is_free"] = gamesdf["is_free"].apply(lambda x : False if x == '0' else True)


# Let's convert the languages into a list of of strings
gamesdf["languages"] = gamesdf["languages"].str.split(",")




### Let's convert the following columns to integers
toIntegerColumnNames = []

# Let's convert the review scores as well as the positive, negative, and total review counts to integers
toIntegerColumnNames.append("review_score") 
toIntegerColumnNames.append("positive")
toIntegerColumnNames.append("negative")
toIntegerColumnNames.append("total")

# Let's convert the metacritic scores to integers too
toIntegerColumnNames.append("metacritic_score")

# Let's convert the recommendation counts to integers
toIntegerColumnNames.append("recommendations")


# Let's convert the steamspy user scores, score ranks, positive review counts, and negative review counts to integers
toIntegerColumnNames.append("steamspy_user_score")
toIntegerColumnNames.append("steamspy_score_rank")
toIntegerColumnNames.append("steamspy_positive")
toIntegerColumnNames.append("steamspy_negative")


# Okay let's convert all of these to integers now! :)
for columnName in toIntegerColumnNames:
    gamesdf[columnName] = gamesdf[columnName].apply(lambda x: int(x) if pd.notna(x) else x)


display(gamesdf.head()) #checking work
print(gamesdf.dtypes) #checking work




### TODO: Use some sort of imputation on missing values/figure out what to do with them


### TODO: Convert the price overview column into multiple columns containing the initial price, final price, and currency???



,app_id,name,release_date,is_free,price_overview,languages,type,genre,review_score,review_score_description,positive,negative,total,metacritic_score,reviews,recommendations,steamspy_user_score,steamspy_score_rank,steamspy_positive,steamspy_negative
0,10,Counter-Strike,2000-11-01,False,"{""final"": 819, ""initial"": 819, ""currency"": ""EU...","[English, French, German, Italian, Spanish...",game,Action,9.0,Overwhelmingly Positive,235403.0,6207.0,241610.0,88.0,NaN,153259.0,0.0,NaN,235397.0,6207.0
1,20,Team Fortress Classic,1999-04-01,False,"{""final"": 499, ""initial"": 499, ""currency"": ""EU...","[English, French, German, Italian, Spanish...",game,Action,8.0,Very Positive,7315.0,1094.0,8409.0,NaN,NaN,6268.0,0.0,NaN,7314.0,1092.0
2,30,Day of Defeat,2003-05-01,False,"{""final"": 499, ""initial"": 499, ""currency"": ""EU...","[English, French, German, Italian, Spanish...",game,Action,8.0,Very Positive,6249.0,672.0,6921.0,79.0,NaN,4146.0,0.0,NaN,6246.0,672.0
3,40,Deathmatch Classic,2001-06-01,False,"{""final"": 499, ""initial"": 499, ""currency"": ""EU...","[English, French, German, Italian, Spanish...",game,Action,8.0,Very Positive,2542.0,524.0,3066.0,NaN,NaN,2218.0,0.0,NaN,2541.0,525.0
4,50,Half-Life: Opposing Force,1999-11-01,False,"{""final"": 499, ""initial"": 499, ""currency"": ""EU...","[English, French, German, Korean]",game,Action,9.0,Overwhelmingly Positive,22263.0,1111.0,23374.0,NaN,NaN,20144.0,0.0,NaN,22260.0,1112.0


app_id                              object
name                                object
release_date                datetime64[ns]
is_free                               bool
price_overview                      object
languages                           object
type                                object
genre                               object
review_score                       float64
review_score_description            object
positive                           float64
negative                           float64
total                              float64
metacritic_score                   float64
reviews                             object
recommendations                    float64
steamspy_user_score                float64
steamspy_score_rank                float64
steamspy_positive                  float64
steamspy_negative                  float64
dtype: object
